In [1]:
import time
import pandas
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common import exceptions
import sys
import io

In [2]:
place = ['홍대 맛집',  '강남 맛집', '신촌 맛집']
review_file = ['result_review(홍대).csv', 'result_review(강남).csv' , 'result_review(신촌).csv']
rest_file = ['result_rest(홍대).csv', 'result_rest(강남).csv' , 'result_rest(신촌).csv']

# 현재 추출해 놓은 데이터 파일들에는 new라는 이름을 붙여놨음.

In [3]:
for vary in range(len(place)) :
    
    driver = webdriver.Chrome(executable_path=r'C:\workspace\chromedriver_win32\chromedriver.exe')
    driver.get("http://www.naver.com")
    #네이버 창 열기
    
    driver.find_element_by_xpath('//*[@id="query"]').clear()
    driver.find_element_by_xpath('//*[@id="query"]').send_keys(place[vary])
    driver.find_element_by_xpath('//*[@id="search_btn"]').click()
    #원하는 맛집카테고리로 검색해서 탭전환
    #현재는 place list에 3곳만 있지만 임의로 추가 가능
    
    driver.switch_to.window(driver.window_handles[-1])
    driver.find_element_by_xpath('//*[@id="place_main_ct"]/div/div/div[3]/a').click()
    driver.switch_to.window(driver.window_handles[-1])
    #검색해서 나오는 카테고리의 모든 플레이스 보이게 탭전환
    
    f = open(review_file[vary], 'w', newline='', encoding = 'utf-8')
    g = open(rest_file[vary], 'w', newline='', encoding = 'utf-8')
    wr = csv.writer(f)
    wg = csv.writer(g)
    wr.writerow(["평점", "리뷰"])
    wg.writerow(["가게이름", "종류", "예약자 리뷰 수", "영수증 리뷰 수"])
   
    page_number = 1
    place_list = [] #중복되는 음식점을 넘어가기 위한 리스트 / 나중을 위해 넣어놓음.
    for p in range (3) : # 5페이지씩 3쪽으로 구성
        if(page_number >= 6) :
            page_number = 1
        for q in range (5) :
            name_list = driver.find_elements_by_class_name('list_item_inner') 
            for name in name_list : # 음식점은 최대 한페이지당 20개
                r_counter = 0
                r2_counter = 0
                driver.implicitly_wait(2)
                try :
                    time.sleep(2)
                    obj = name.find_element_by_class_name('name')
                    driver.execute_script("arguments[0].click();",obj)
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(2)
                    # 음식점 창으로 들어가서 탭전환 , 리뷰는 최대 한 페이지당 10개
                    # 예약자,영수증 리뷰 없이 블로그리뷰만 있는경우는 리뷰 수집 X
                    flag = 0
                    temp_place = [0,0,0,0]
                    place_emem = driver.find_element_by_class_name('biz_name_area')
                    print(place_emem.find_element_by_class_name('name').text)
                    for k in place_list : # 중복되는 음식점은 넘어감
                        if (k[0] == place_emem.find_element_by_class_name('name').text) :
                            flag = 1 # 넘어가기 위한 flag
                    if (flag == 1) :
                        driver.close()
                        driver.switch_to.window(driver.window_handles[-1])
                        time.sleep(2)
                        continue
                    temp_place[0]=place_emem.find_element_by_class_name('name').text
                    temp_place[1]=place_emem.find_element_by_class_name('category').text
                    # 음식점 이름 , 음식점 카테고리 저장

                    tab_object = driver.find_element_by_xpath('//*[@id="tab03"]')
                    tab_check = tab_object.get_attribute("aria-label")
                    if ( tab_check == "예약자 리뷰") :
                        driver.execute_script("arguments[0].click();",tab_object)
                        
                        time.sleep(1)
                        #예약자 리뷰 수
                        # 예약자 리뷰 창 들어감, index는 리뷰 페이지 수
                        try : #리뷰페이지 수가 2 이상인 경우
                            index = int(driver.find_element_by_class_name('total').text)
                        except : #리뷰페이지 수가 1인 경우 예외처리
                            index = 1
                        for i in range (index) :
                            star_list = []
                            star_text = []
                            review_list = []
                            review_text = [] 
                            star_list = driver.find_elements_by_class_name('star_area')
                            review_list = driver.find_elements_by_class_name('review_txt')
                            count = len(star_list)
                            for k in range (1,count) : 
                                star_text.append(star_list[k].find_element_by_class_name('score').text)
                                review_text.append(review_list[k-1].text)
                                wr.writerow([star_text[k-1], review_text[k-1]])
                                r_counter += 1
                                driver.implicitly_wait(0.3)
                            if ( index > 1) :
                                if ( index - i != 1) :
                                        if ( i <= 20) :
                                            # 리뷰가 너무 많은 식당은 전부 뽑지 않고, 일부만 뽑는다. i <= 20
                                            obj = driver.find_element_by_xpath('//*[@id="panel03"]/div/div[4]/a[2]')
                                            driver.execute_script("arguments[0].click();",obj)
                                            time.sleep(2)
                                        else :
                                            continue

                    elif ( tab_check == "영수증 리뷰") :
                        driver.execute_script("arguments[0].click();",tab_object)
                        
                        try :
                            index = int(driver.find_element_by_class_name('total').text)
                        except :
                            index = 1
                        for i in range (index) :
                            star_list = []
                            star_text = []
                            review_list = []
                            review_text = []
                            star_list = driver.find_elements_by_class_name('star_area')
                            review_list = driver.find_elements_by_class_name('review_txt')
                            count = len(star_list)
                            for k in range (0,count) : 
                                star_text.append(star_list[k].find_element_by_class_name('score').text)
                                review_text.append(review_list[k].text)
                                wr.writerow([star_text[k], review_text[k]])
                                r2_counter += 1
                                time.sleep(0.2)
                            if ( index > 1) :
                                if ( index - i != 1) :
                                    if ( i <= 20) :
                                        obj = driver.find_element_by_xpath('//*[@id="panel03"]/div/div[2]/a[2]')
                                        driver.execute_script("arguments[0].click();",obj)
                                        time.sleep(2)
                                    else :
                                        continue

                    try : # xpath 에서 tab04에 영수증리뷰가 있는경우
                        tab_object = driver.find_element_by_xpath('//*[@id="tab04"]')
                        tab_check = tab_object.get_attribute("aria-label")
                        if ( tab_check == "영수증 리뷰") :
                            driver.execute_script("arguments[0].click();",tab_object)
                            
                            try :
                                index = int(driver.find_element_by_class_name('total').text)
                            except :
                                index = 1
                            for i in range (index) :
                                star_list = []
                                star_text = []
                                review_list = []
                                review_text = []
                                star_list = driver.find_elements_by_class_name('star_area')
                                review_list = driver.find_elements_by_class_name('review_txt')
                                count = len(star_list)
                                for k in range (0,count) : 
                                    star_text.append(star_list[k].find_element_by_class_name('score').text)
                                    review_text.append(review_list[k].text)
                                    wr.writerow([star_text[k], review_text[k]])
                                    r2_counter += 1
                                    time.sleep(0.2)
                                if ( index > 1) :
                                    if ( index - i != 1) :
                                        if ( i <= 20) :
                                            obj = driver.find_element_by_xpath('//*[@id="panel04"]/div/div[2]/a[2]')
                                            driver.execute_script("arguments[0].click();",obj)
                                            time.sleep(2)
                                        else :
                                            continue
                    except : print('')# # xpath 에서 tab04에 영수증리뷰가 없는경우 예외처리.  no action
                        
                    temp_place[2] = r_counter
                    temp_place[3] = r2_counter
                    place_list.append(temp_place)
                    wg.writerow(temp_place)
                    time.sleep(2)
                    driver.close()
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(5)
                except exceptions.StaleElementReferenceException as e:
                    time.sleep(10)
            page_number = page_number+1
            # 음식점 20개를 모두 확인 한 뒤, 페이지 넘김
            if(page_number == 2) :     
                obj = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div/div[2]/div/div[2]/a[3]')
                driver.execute_script("arguments[0].click();",obj)
                driver.implicitly_wait(2)
            elif( page_number == 3) :
                obj=driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div/div[2]/div/div[2]/a[4]')
                driver.execute_script("arguments[0].click();",obj)
                driver.implicitly_wait(2)
            elif( page_number == 4) :
                obj=driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div/div[2]/div/div[2]/a[5]')
                driver.execute_script("arguments[0].click();",obj)
                driver.implicitly_wait(2)
            elif( page_number == 5) :
                obj=driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div/div[2]/div/div[2]/a[6]')
                driver.execute_script("arguments[0].click();",obj)
                driver.implicitly_wait(2)
            else : break
        obj = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[1]/div/div[2]/div/div[2]/a[7]')
        driver.execute_script("arguments[0].click();",obj)
        driver.implicitly_wait(2)
    f.close()
    g.close()
    print(place[vary] + '끝') #체크용 문구
    # 홍대 -> 신촌 -> 강남 으로 넘어가야됨
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    driver.close()
    time.sleep(2)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 78


In [18]:
#문서 확인용 코드
import pandas as pd
f.close()
g.close()


In [22]:
dataset = pd.read_csv("result_rest(홍대new).csv", encoding = 'utf-8')
display(dataset)

,가게이름,종류,예약자 리뷰 수,영수증 리뷰 수
0,쏘굿홍대소고기무한리필,소고기구이,187,0
1,한양집 홍대점,한식,54,19
2,감성타코 홍대점,"멕시코,남미음식",270,11
3,여우골 홍대점,"초밥,롤",0,4
4,바다회사랑2호점,생선회,0,15
5,부야스곱창,"곱창,막창,양",0,14
6,로쏘 1924,피자,0,7
7,혼가츠,돈가스,0,9
8,델리인디아,인도음식,880,6
9,테이스티버거,햄버거,0,27


In [23]:
dataset = pd.read_csv("result_review(홍대new).csv", encoding = 'utf-8')
display(dataset)

,평점,리뷰
0,4,예약했는ㄷ데 음료 서비스 안주시던데요. 고기는 맛있게 잘 먹었는데 우삼겹 먹는 방법...
1,3,그양그랫러요
2,3,그냥 그랫어요
3,5,굿 직원분들 친절하고 빠릿합니다 고기도 가성비 굿이구요 다만 반찬셋팅하고난후 숯불 ...
4,4,고기도 괜찮고 밑반찬도 괜찮네요. 근데 사람이 많으니 좀 시끄럽고 직원들 움직이면서...
5,5,정말 맛있게 잘먹었습니다 특히 사장님이 열심히 하시는게 인상적이였습니다 자주 먹으러...
6,5,맛있었어요^^
7,3,서비스가좀더잘되었음좋겠네요.
8,4,직원들도 친절하고 고기도 맛있고 즐거웠습니다 대박나세요~~
9,5,고기 질이 좋았습니다.
